# **최적화(Optimization)**


* 목적함수(Object Function)을 최대/최소화 하는 파라미터 조합 탐색 문제
* 머신 러닝에서는 일반적으로 모델이 데이터를 잘 예측하거나 분류할 수 있도록 손실 함수(loss function)를 최소화하는 것이 목표

In [ ]:
#GPU 사용 확인
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

## **0. 데이터 준비**
* Fashion MNIST
* 10개의 의류 품목(셔츠, 바지 등)으로 구성된 28x28 픽셀의 흑백 이미지 데이터셋

In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import imgaug.augmenters as iaa
from PIL import Image

### **데이터셋 경로 설정**

In [ ]:
original_data_dir = "fashion_mnist_images"  
output_base_dir = "fashion_mnist_dataset"
train_dir = os.path.join(output_base_dir, "train")
valid_dir = os.path.join(output_base_dir, "valid")
test_dir = os.path.join(output_base_dir, "test")

In [ ]:
for class_name in os.listdir(original_data_dir):
    print(class_name)

### **데이터셋 분할: 0.7 (Train), 0.2 (Validation), 0.1 (Test)**

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

def split_data_stratified(original_dir, train_ratio=0.7, valid_ratio=0.2):
    # 모든 데이터를 모으고 라벨 생성
    image_paths = []
    labels = []
    for class_name in os.listdir(original_dir):
        class_dir = os.path.join(original_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            if img_name.endswith(('.png', '.jpg')):
                image_paths.append(os.path.join(class_dir, img_name))
                labels.append(class_name)

    # 데이터 분할
    train_images, temp_images, train_labels, temp_labels = train_test_split(
        image_paths, labels, test_size=(1 - train_ratio), stratify=labels, random_state=42
    )
    valid_size = valid_ratio / (1 - train_ratio)  # validation 비율 조정
    valid_images, test_images, valid_labels, test_labels = train_test_split(
        temp_images, temp_labels, test_size=(1 - valid_size), stratify=temp_labels, random_state=42
    )

    # 데이터를 출력 디렉토리에 저장
    for dataset, output_dir in zip(
        [(train_images, train_labels), (valid_images, valid_labels), (test_images, test_labels)],
        [train_dir, valid_dir, test_dir]
    ):
        images, labels = dataset
        for img_path, label in zip(images, labels):
            class_output_dir = os.path.join(output_dir, label)
            os.makedirs(class_output_dir, exist_ok=True)
            shutil.copy(img_path, class_output_dir)


In [ ]:
split_data_stratified(original_data_dir)

## **1. 과적합(Overfitting)**
* 모델이 학습 데이터에 대한 성능은 매우 우수하지만, 새로운 데이터에 대해서는 일반화 성능이 떨어지는 현상

<img src="Contents/Fitting.png" alt=" Frtting" width="1000" height="400"/>

## **1-2. 과적합 해결방안: 데이터 증강(Data Augmentation)**
* 데이터의 양이 적을 경우, 해당 데이터의 특정 패턴이나 노이즈까지 학습
* 데이터의 양을 늘릴 수록 모델은 데이터의 일반적인 패턴을 학습하여 과적합 방지
* Train Data에 대해서만 데이터 증강 수행(Data Augmentation)

### **데이터 증강(Data Augmentation)**

In [ ]:
import os
import imgaug.augmenters as iaa
from PIL import Image

In [ ]:
#imgaug의 iaa.Sequential을 사용하여 다양한 증강 기법을 적용
def augment_images(input_dir, output_dir):
    
    # Augmentation 설정
    seq = iaa.Sequential([
        iaa.Fliplr(0.5),  # 수평 반전
        #iaa.Flipud(0.5),  # 수직 반전
        iaa.Affine(rotate=(-15, 15)),  # -15도에서 15도까지 회전
        #iaa.Multiply((0.8, 1.2)),  # 밝기 조정
        #iaa.GaussianBlur(sigma=(0, 1.0))  # 가우시안 Blur
    ])

    class_dirs = [os.path.join(input_dir, class_name) for class_name in os.listdir(input_dir)]
    for class_dir in class_dirs:
        if not os.path.isdir(class_dir):
            continue

        output_class_dir = os.path.join(output_dir, os.path.basename(class_dir))
        os.makedirs(output_class_dir, exist_ok=True)

        # 이미지를 읽어서 증강 후 저장
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            if not img_path.endswith(('.png', '.jpg')):
                continue

            # 이미지를 불러와 증강
            image = Image.open(img_path)
            image_np = np.array(image)
            augmented_images = seq(images=[image_np] * 1)  # 이미지 1개 생성

            for i, aug_img in enumerate(augmented_images):
                aug_img = Image.fromarray(aug_img)
                aug_img.save(os.path.join(output_class_dir, f"{img_name.split('.')[0]}_aug_{i}.png"))

In [ ]:
input_train='fashion_mnist_dataset/train'
output_train='augmented_fashion_mnist_dataset/train'
augment_images(input_train, output_train)

### **증강된 이미지 확인**

In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt

# 증강된 데이터 경로 설정
augmented_data_dir = "augmented_fashion_mnist_dataset/train"  # 증강 데이터가 저장된 디렉토리
class_name = "Ankle boot"  # 확인할 클래스 이름
class_dir = os.path.join(augmented_data_dir, class_name)

# 특정 클래스 디렉토리에서 이미지 읽기
images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('.png', '.jpg'))]

# 확인할 이미지 선택
num_images_to_display = 5  # 확인할 이미지 수
selected_images = images[:num_images_to_display]

# 이미지 시각화
plt.figure(figsize=(12, 6))
for i, img_path in enumerate(selected_images):
    # 파일 이름 추출
    image_name = os.path.basename(img_path)
    
    # PIL로 이미지 열기
    image = Image.open(img_path)
    
    # 시각화
    plt.subplot(1, num_images_to_display, i + 1)
    plt.imshow(image)
    plt.axis("off")
    plt.title(image_name)  # 이미지 파일 이름을 제목으로 표시
    
    # 파일 이름 출력
    print(f"Displaying: {image_name}")

plt.tight_layout()
plt.show()

### **증강 전후 결과 비교**

In [ ]:
# 데이터 경로 설정
original_train_dir = "fashion_mnist_dataset/train"
augmented_train_dir = "augmented_fashion_mnist_dataset/train"
valid_dir = "fashion_mnist_dataset/valid"  # 증강 후에도 동일한 validation 사용
test_dir = "fashion_mnist_dataset/test"

In [ ]:
# 이미지 크기 설정 및 신경망 입력 크기
image_size = (28, 28)  # Fashion MNIST는 28x28 크기
input_size = image_size[0] * image_size[1]

### **CNN 모델 생성**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# CNN 모델 정의
def build_cnn_model():
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1))) # Convolutional Layer: 32개의 필터, 커널 크기 3x3, 활성화 함수 ReLU
    model.add(MaxPooling2D(pool_size=(2, 2)))# Max Pooling Layer: 2x2 풀링

    # Convolutional Layer: 64개의 필터, 커널 크기 3x3, 활성화 함수 ReLU
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu')) # Convolutional Layer: 64개의 필터, 커널 크기 3x3, 활성화 함수 ReLU
    model.add(MaxPooling2D(pool_size=(2, 2)))# Max Pooling Layer: 2x2 풀링
    model.add(Flatten()) # Flatten Layer: 2D 데이터를 1D 벡터로 변환

    
    model.add(Dense(units=128, activation='relu')) # Fully Connected Layer: 128 뉴런, 활성화 함수 ReLU
    model.add(Dense(units=10, activation='softmax')) # Output Layer: 10개의 클래스, 활성화 함수 Softmax

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy', 
        optimizer=Adam(learning_rate=0.001), 
        metrics=['accuracy']
    )

    return model

### **Data 전처리**

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder

def load_data(data_dir, image_size=(28, 28)):
    images = []
    labels = []
    
    # 디렉토리 구조 탐색
    for class_name in sorted(os.listdir(data_dir)):  # 클래스는 디렉토리 이름
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):  # 디렉토리가 아니면 건너뜀
            continue
        
        # 각 클래스의 이미지 탐색
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            if img_path.endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일만 처리
                # 이미지 로드 및 전처리
                img = Image.open(img_path).convert('L')  # 그레이스케일 변환
                img = img.resize(image_size)  # 크기 조정
                img = np.array(img) / 255.0  # 정규화 (0~1)
                images.append(img.flatten())  # 플래튼 (784,)
                labels.append(class_name)  # 라벨 추가
    
    # NumPy 배열로 변환
    images = np.array(images, dtype=np.float32) 
    labels = np.array(labels)
    
    # 라벨을 정수형으로 변환
    le = LabelEncoder()
    labels = le.fit_transform(labels)  # 문자열 라벨 → 정수 라벨로 변환
    
    return images, labels

In [ ]:
# 데이터 로드
X_train, y_train = load_data(original_train_dir)
X_train_aug, y_train_aug = load_data(augmented_train_dir)
X_train_combined = np.concatenate([X_train, X_train_aug], axis=0)
y_train_combined = np.concatenate([y_train, y_train_aug], axis=0)
X_valid, y_valid = load_data(valid_dir)
X_test, y_test = load_data(test_dir)

### **CNN 모델 학습 및 평가 함수**

In [ ]:
# CNN 모델 학습 및 평가 함수
def train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model, epochs, description):
    print(f"\nTraining with {description} data...\n")

    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)

    # 모델 학습
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=8,
        verbose=1
    )

    # Test 데이터 성능 평가
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy ({description}): {test_accuracy * 100:.2f}%\n")

    return history

### **학습 결과 시각화**

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history, description):
    # 학습 및 검증 손실 곡선
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{description} - Loss')
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # 학습 및 검증 정확도 곡선
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{description} - Accuracy')
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

### **원본 데이터만 학습한 모델**

In [ ]:
# 학습 곡선 시각화
model=build_cnn_model()
history_orig = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model, 10, "Original")
plot_training_history(history_orig, "Original Data")

### **증강된 데이터를 포함해 학습한 모델**

In [ ]:
model=build_cnn_model()
history_aug = train_and_evaluate_cnn_model(X_train_combined, y_train_combined, X_valid, y_valid, X_test, y_test, model, 10, "Augmented")
plot_training_history(history_aug, "Augmented Data")

## **1-3. 과적합 해결방안: 모델 복잡도 낮춤**
* 모델이 훈련 데이터의 노이즈까지 학습하게 되어 테스트 데이터나 새로운 데이터에 대한 일반화 성능 감소 
* 인공 신경망의 복잡도는 은닉층(hidden layer)의 수나 매개변수의 수 등으로 결정

## **1-4. 과적합 해결방안: 드롭아웃(Dropout)**
* 일정 비율의 가중치를 임의로 선택하여 불능으로 만들고 학습하는 규제 기법
* 불능이 될 엣지는 샘플마다 독립적으로 난수를 이용하여 랜덤하게 선택

<img src="Contents/dropout.png" alt=" Dropout" width="600" height="300"/>

### **Dropout 적용하지 않은 모델**

In [ ]:
def build_cnn_model_no_dropout(input_shape=(28, 28, 1), num_classes=10):
    model = Sequential()

    # 첫 번째 Convolutional Layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 Convolutional Layer
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(units=512, activation='relu'))

    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    return model

### **Dropout 적용한 모델**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# CNN 모델 정의
def build_cnn_model_dropout(input_shape=(28, 28, 1), num_classes=10, dropout_rate=[0.25, 0.25, 0.5]):
    model = Sequential()

    # 첫 번째 Convolutional Layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=dropout_rate[0]))

    # 두 번째 Convolutional Layer
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=dropout_rate[1]))

    # Flatten Layer
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(rate=dropout_rate[2]))

    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy', 
        optimizer=Adam(learning_rate=0.001), 
        metrics=['accuracy']
    )

    return model


### **모델 비교 평가 수행**

In [ ]:
# Dropout 적용 전후 비교 학습
def compare_dropout_models(X_train, y_train, X_valid, y_valid, X_test, y_test):
    # Dropout 없는 모델
    print("\nTraining CNN Model WITHOUT Dropout...\n")
    no_dropout_model = build_cnn_model_no_dropout()
    X_train_no_dropout = X_train.reshape(-1, 28, 28, 1)
    X_valid_no_dropout = X_valid.reshape(-1, 28, 28, 1)
    X_test_no_dropout = X_test.reshape(-1, 28, 28, 1)
    history_no_dropout = no_dropout_model.fit(
        X_train_no_dropout, y_train,
        validation_data=(X_valid_no_dropout, y_valid),
        epochs=10,
        batch_size=8,
        verbose=1
    )
    test_loss_no_dropout, test_accuracy_no_dropout = no_dropout_model.evaluate(X_test_no_dropout, y_test, verbose=0)
    print(f"\nTest Accuracy WITHOUT Dropout: {test_accuracy_no_dropout * 100:.2f}%\n")

    # Dropout 있는 모델
    print("\nTraining CNN Model WITH Dropout...\n")
    dropout_model = build_cnn_model_dropout()
    X_train_dropout = X_train.reshape(-1, 28, 28, 1)
    X_valid_dropout = X_valid.reshape(-1, 28, 28, 1)
    X_test_dropout = X_test.reshape(-1, 28, 28, 1)
    history_dropout = dropout_model.fit(
        X_train_dropout, y_train,
        validation_data=(X_valid_dropout, y_valid),
        epochs=10,
        batch_size=8,
        verbose=1
    )
    test_loss_dropout, test_accuracy_dropout = dropout_model.evaluate(X_test_dropout, y_test, verbose=0)
    print(f"\nTest Accuracy WITH Dropout: {test_accuracy_dropout * 100:.2f}%\n")

    # 결과 시각화
    print("Plotting results...")
    plot_training_history(history_no_dropout, "CNN WITHOUT Dropout")
    plot_training_history(history_dropout, "CNN WITH Dropout")

In [ ]:
# 데이터 로드
X_train, y_train = load_data(original_train_dir)
X_valid, y_valid = load_data(valid_dir)
X_test, y_test = load_data(test_dir)

# Dropout 적용 전후 모델 비교
compare_dropout_models(X_train, y_train, X_valid, y_valid, X_test, y_test)

## **1-5. 과적합 해결방안: 조기 종료(Early Stop)**
* 모델이 훈련 데이터에서 너무 오랜 시간 동안 학습하면, 학습 데이터에 과적합되어 테스트 데이터나 새로운 데이터에 대한 일반화 성능 감소
* 모델이 과적합되기 전에 학습을 멈추는 방법

<img src="Contents/Early_stop.png" alt="Early Stop" width="600" height="400"/>

### **Early Stop 적용한 모델**

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

def train_and_evaluate_cnn_model_with_early_stopping(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs, patience,description):
    print(f"\nTraining with {description} data and Early Stopping...\n")

    # CNN 모델 생성
    model = build_cnn_model()

    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)

    # Early Stopping 콜백 추가
    early_stopping = EarlyStopping(
        monitor='val_loss',  # 검증 손실을 기준으로 관찰
        patience=patience,          # 개선되지 않은 에포크 수 (patience 에포크 동안 개선 없으면 멈춤)
        restore_best_weights=True  # 최적의 가중치 복원
    )

    # 모델 학습
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,  # 최대 에포크 수
        batch_size=8,
        verbose=1,
        callbacks=[early_stopping]
    )

    # Test 데이터 성능 평가
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy ({description}): {test_accuracy * 100:.2f}%\n")

    return history

### **모델 비교 평가 수행**

In [ ]:
def compare_early_stopping(X_train, y_train, X_valid, y_valid, X_test, y_test):
    # Early Stopping 없이 학습
    print("Training CNN Model WITHOUT Early Stopping...\n")
    model = build_cnn_model()
    history_no_es = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test,model, 50 ,"CNN WITHOUT Early Stopping")

    # Early Stopping 적용 학습
    print("Training CNN Model WITH Early Stopping...\n")
    history_with_es = train_and_evaluate_cnn_model_with_early_stopping(X_train, y_train, X_valid, y_valid, X_test, y_test, 50, 5, "CNN WITH Early Stopping")

    # 결과 시각화
    print("Plotting results...")
    plot_training_history(history_no_es, "CNN WITHOUT Early Stopping")
    plot_training_history(history_with_es, "CNN WITH Early Stopping")

In [ ]:
# 데이터 로드
X_train, y_train = load_data(original_train_dir)
X_valid, y_valid = load_data(valid_dir)
X_test, y_test = load_data(test_dir)

# Early Stopping 전후 비교
compare_early_stopping(X_train, y_train, X_valid, y_valid, X_test, y_test)

## **1-6. 과적합 해결방안: 가중치 규제(Regularization)**
* 모델의 과적합을 방지하고 일반화 성능을 향상시키기 위해 가중치의 크기를 제한
* 모델이 복잡해지는 것을 억제하고, 과적합 감소시킴
* L1 정규화(L1 Regularization)
    * 가중치의 절대값 합을 최소화
    * 일부 가중치를 정확히 0으로 만듦
    * 모델을 더 희소(sparse)하게 만들어 불필요한 특성을 제거
* L2 정규화(L2 Regularization)
    * 가중치의 제곱합을 최소화
    * Weight Decay:딥러닝 모델의 가중치 크기를에 L2 정규화 항을 규제하여 과적합을 방지하는 정규화 기법

<img src="Contents/l1l2.png" alt="Early Stop" width="600" height="400"/>

### **규제가 없는 모델** 

In [ ]:
# CNN 모델 학습 및 평가 함수
def train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model, epochs, description,):
    print(f"\nTraining with {description} data...\n")


    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)

    # 모델 학습
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=32,
        verbose=1
    )

    # Test 데이터 성능 평가
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy ({description}): {test_accuracy * 100:.2f}%\n")

    return history

### **L1 Regularization** 

In [ ]:
from tensorflow.keras.regularizers import l1

def build_cnn_model_l1(input_shape=(28, 28, 1), num_classes=10, l1_rate=0.01):
    model = Sequential()

    # 첫 번째 Convolutional Layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape,
                     kernel_regularizer=l1(l1_rate)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 Convolutional Layer
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=l1(l1_rate)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(units=128, activation='relu', kernel_regularizer=l1(l1_rate)))

    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    return model

### **L2 Regularization**

In [ ]:
from tensorflow.keras.regularizers import l2

def build_cnn_model_l2(input_shape=(28, 28, 1), num_classes=10, l2_rate=0.01):
    model = Sequential()

    # 첫 번째 Convolutional Layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape,
                     kernel_regularizer=l2(l2_rate)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 Convolutional Layer
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',
                     kernel_regularizer=l2(l2_rate)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(units=128, activation='relu', kernel_regularizer=l2(l2_rate)))

    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    return model

### **모델 비교 평가 수행**

In [ ]:
def compare_regularization_models(X_train, y_train, X_valid, y_valid, X_test, y_test):
    # Regularization 없이 학습
    print("Training CNN Model WITHOUT Regularization...\n")
    model=build_cnn_model()
    history_no_reg = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model,20,"CNN WITHOUT Regularization")

    # L1 Regularization 적용
    print("Training CNN Model WITH L1 Regularization...\n")
    model_l1 = build_cnn_model_l1()
    history_l1 = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model_l1,20,"CNN WITH L1 Regularization")

    # L2 Regularization 적용
    print("Training CNN Model WITH L2 Regularization...\n")
    model_l2 = build_cnn_model_l2()
    history_l2 = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test,model_l2,20 ,"CNN WITH L2 Regularization")

    # 결과 시각화
    print("Plotting results...")
    plot_training_history(history_no_reg, "CNN WITHOUT Regularization")
    plot_training_history(history_l1, "CNN WITH L1 Regularization")
    plot_training_history(history_l2, "CNN WITH L2 Regularization")

In [ ]:
# 데이터 로드
X_train, y_train = load_data(original_train_dir)
X_valid, y_valid = load_data(valid_dir)
X_test, y_test = load_data(test_dir)

# Regularization 비교
compare_regularization_models(X_train, y_train, X_valid, y_valid, X_test, y_test)

## **1-6. 과적합 해결방안: 배치 정규화(Batch Normalization)**
* 배치(Batch):전체 데이터셋을 여러 작은 그룹으로 나눈 데이터 묶음으로, 한 번의 학습(iteration)에서 처리되는 데이터 단위
* 학습 과정에서 각 배치 단위 별 다양한 분포를 가진 데이터를 각 배치별로 평균과 분산을 이용해 정규화
* 모델이 특정 입력값의 스케일이나 분포에 덜 민감해지므로, 네트워크가 데이터를 지나치게 복잡하게 학습(overfitting)하는 것을 방지

<img src="Contents/BatchNormalization.png" alt="Early Stop" width="600" height="400"/>

### **배치 정규화를 적용하지 않은 모델**

In [ ]:
from tensorflow.keras.layers import BatchNormalization

# Batch Normalization이 없는 CNN 모델
def build_cnn_model_without_bn(input_shape=(28, 28, 1), num_classes=10):
    model = Sequential()

    # 첫 번째 Convolutional Layer
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 Convolutional Layer
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(units=128, activation='relu'))

    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    return model

### **배치 정규화를 적용한 모델**

In [ ]:
# Batch Normalization이 포함된 CNN 모델
def build_cnn_model_with_bn(input_shape=(28, 28, 1), num_classes=10):
    model = Sequential()

    # 첫 번째 Convolutional Layer + Batch Normalization
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # 두 번째 Convolutional Layer + Batch Normalization
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten Layer
    model.add(Flatten())

    # Fully Connected Layer + Batch Normalization
    model.add(Dense(units=128, activation='relu'))
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(units=num_classes, activation='softmax'))

    # 모델 컴파일
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    return model

### **모델 비교 평가 수행**

In [ ]:
# Batch Normalization 비교 함수
def compare_batch_normalization_models(X_train, y_train, X_valid, y_valid, X_test, y_test):
    # Batch Normalization 없이 학습
    print("Training CNN Model WITHOUT Batch Normalization...\n")
    model_without_bn = build_cnn_model_without_bn()
    history_without_bn = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model_without_bn, 10, "CNN WITHOUT Batch Normalization")

    # Batch Normalization 포함 학습
    print("Training CNN Model WITH Batch Normalization...\n")
    model_with_bn = build_cnn_model_with_bn()
    history_with_bn = train_and_evaluate_cnn_model(X_train, y_train, X_valid, y_valid, X_test, y_test, model_with_bn, 10, "CNN WITH Batch Normalization")

    # 결과 시각화
    print("Plotting results...")
    plot_training_history(history_without_bn, "CNN WITHOUT Batch Normalization")
    plot_training_history(history_with_bn, "CNN WITH Batch Normalization")



In [ ]:
# 데이터 로드
X_train, y_train = load_data(original_train_dir)
X_valid, y_valid = load_data(valid_dir)
X_test, y_test = load_data(test_dir)

# Batch Normalization 비교
compare_batch_normalization_models(X_train, y_train, X_valid, y_valid, X_test, y_test)


## **2. 최적화 알고리즘 (Optimizer)**
* 딥러닝 모델의 손실 함수를 최소화하기 위해 가중치(weight)와 편향(bias)을 효율적으로 업데이트하는 알고리즘

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD,Adam,Adagrad,RMSprop

### **2-1.확률적 경사 하강법(Stochastic Gradient Descent)**
* 확률적으로 데이터를 뽑아 단일 데이터 포인트 또는 작은 배치를 사용하여 가중치들을 업데이트하는 방법
* 전체 데이터를 사용하는 경사하강법과 달리 SGD는 연산량이 비교적 매우 적어 손실 함수의 최적값에 빠르게 수렴

<img src="Contents/SGD.png" alt=" Frtting" width="500" height="200"/>
<img src="Contents/EQ_SGD.png" alt=" Frtting" width="500" height="200"/>

In [ ]:
def train_with_sgd(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs):
    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)
    
    print("\nTraining with SGD...\n")
    model = build_cnn_model()
    optimizer = SGD(learning_rate=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=8,
        verbose=1
    )
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy (SGD): {test_accuracy * 100:.2f}%\n")
    return history

In [ ]:
epochs=10

In [ ]:
history_sgd = train_with_sgd(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs)
plot_training_history(history_sgd, "SGD")

### **2-2. 모멘텀(Momentum)**
* SGD에 관성을 추가한 기법으로, 이전 단계의 기울기 방향을 일부 유지하여 진동을 줄이고 빠르게 수렴

<img src="Contents/Momentum.png" alt=" Frtting" width="500" height="200"/> 
<img src="Contents/EQ_Momentum.png" alt=" Frtting" width="500" height="200"/>

In [ ]:
def train_with_momentum(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs):
    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)
    
    print("\nTraining with SGD + Momentum...\n")
    model = build_cnn_model()
    optimizer = SGD(learning_rate=0.01, momentum=0.9)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=8,
        verbose=1
    )
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy (SGD + Momentum): {test_accuracy * 100:.2f}%\n")
    return history

In [ ]:
history_momentum = train_with_momentum(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs)
plot_training_history(history_momentum, "SGD with Momentum")

### **2-3. 적응적 학습률(Adaptive Learning rate)**
* 학습 과정에서 각 파라미터의 업데이트 크기를 자동으로 조정하여 학습률을 동적으로 변화시키는 방법

#### **2-3-1. AdaGrad(Adaptive Gradient)**
* 각 가중치의 학습률을 개별적으로 조정하여 많이 업데이트된 가중치 학습률은 작게, 덜 업데이트된 가중치는 크게 조정하여 학습
* 학습률이 너무 작아져 학습이 멈출 수 있음

<img src="Contents/AdaGrad.png" alt=" Frtting" width="500" height="200"/> 
<img src="Contents/EQ_AdaGrad.png" alt=" Frtting" width="500" height="200"/>

In [ ]:
def train_with_adagrad(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs):
    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)
    
    print("\nTraining with AdaGrad...\n")
    model = build_cnn_model()
    optimizer = Adagrad(learning_rate=0.01)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=8,
        verbose=1
    )
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy (AdaGrad): {test_accuracy * 100:.2f}%\n")
    return history

In [ ]:
history_adagrad = train_with_adagrad(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs)
plot_training_history(history_adagrad, "AdaGrad")

#### **2-3-2. RMSProp(Root Mean Sqaure Propagation)**
* AdaGrad의 문제를 개선한 방법으로, 기울기 제곱의 이동 평균을 사용하여 학습률을 조정
* 먼 과거의 기울기는 조금 반영하고 최신의 기울기를 많이 반영
* 학습률이 지나치게 감소하는 문제를 해결하고, 수렴 속도를 개선

<img src="Contents/EQ_RMSProp.png" alt=" Frtting" width="500" height="200"/>

In [ ]:
def train_with_rmsprop(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs):
    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)
    
    print("\nTraining with RMSProp...\n")
    model = build_cnn_model()
    optimizer = RMSprop(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=8,
        verbose=1
    )
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy (RMSProp): {test_accuracy * 100:.2f}%\n")
    return history

In [ ]:
history_rmsprop = train_with_rmsprop(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs)
plot_training_history(history_rmsprop , "RMSProp")

#### **2-3-3. Adam(Adaptive Moment Estimation)**
* Momentum과 RMSProp을 결합한 최적화 알고리즘

<img src="Contents/Adam.png" alt=" Frtting" width="500" height="200"/>

In [ ]:
def train_with_adam(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs):
    # 데이터 형태 변환: CNN 입력에 맞게 4D 텐서로 변환
    X_train = X_train.reshape(-1, 28, 28, 1)  # (샘플 수, 높이, 너비, 채널 수)
    X_valid = X_valid.reshape(-1, 28, 28, 1)
    X_test = X_test.reshape(-1, 28, 28, 1)
    
    print("\nTraining with Adam...\n")
    model = build_cnn_model()
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=epochs,
        batch_size=8,
        verbose=1
    )
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy (Adam): {test_accuracy * 100:.2f}%\n")
    return history

In [ ]:
history_adam = train_with_adam(X_train, y_train, X_valid, y_valid, X_test, y_test, epochs)
plot_training_history(history_rmsprop , "Adam")

## **3. 하이퍼 파라미터 튜닝(Hyperparameter Tuning)**
* 머신러닝 모델의 성능을 최적화하기 위해 모델 학습 과정에서 고정된 값을 가지는 하이퍼파라미터를 조정하는 과정
* 주요 하이퍼파라미터로는 학습률(learning rate), 정규화 파라미터, 은닉층의 수와 크기 등
* Keras-Tuner를 사용하여 딥러닝 모델 하이퍼파라미터 튜닝

### **3-1. 그리드 서치(Grid Search)**
* 그리드 서치는 사전에 정의된 하이퍼파라미터 값의 조합을 모두 탐색하여 최적의 파라미터를 찾는 방법

![Local Image](Contents/Grid_Search.png)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from kerastuner.tuners import GridSearch

# CNN 모델 생성 함수
def build_model(hp):
    model = Sequential()
    # 첫 번째 Convolutional Layer
    model.add(Conv2D(
        filters=hp.Choice('filters1', values=[32, 64]), 
        kernel_size=(3, 3), 
        activation='relu', 
        input_shape=(28, 28, 1)))
    model.add(Conv2D(
        filters=hp.Choice('filters2', values=[64, 128]), 
        kernel_size=(3, 3), 
        activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout1', values=[0.2, 0.3])))

    # 두 번째 Convolutional Layer
    model.add(Conv2D(
        filters=hp.Choice('filters3', values=[128, 256]), 
        kernel_size=(3, 3), 
        activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout2', values=[0.3, 0.4])))

    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(hp.Choice('dense_units', values=[128, 256]), activation='relu'))
    model.add(Dropout(hp.Choice('dropout3', values=[0.4, 0.5])))
    model.add(Dense(10, activation='softmax'))  # 10개의 클래스

    # Optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Keras Tuner Grid Search 설정
tuner = GridSearch(
    build_model,
    objective='val_accuracy',  # 최적화 목표
    max_trials=20,            # 시도할 하이퍼파라미터 조합 수
    executions_per_trial=1,   # 각 조합 실행 횟수
    directory='my_dir',       # 결과 저장 디렉토리
    project_name='cnn_grid_tuning' # 프로젝트 이름
)

# 데이터 준비
X_train_reshaped = X_train_combined.reshape(-1, 28, 28, 1)  # 4D 텐서로 변환
X_valid_reshaped = X_valid.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)

# Grid Search 실행
tuner.search(X_train_reshaped, y_train_combined, 
             validation_data=(X_valid_reshaped, y_valid), 
             epochs=10, 
             batch_size=32)

# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# 최적 모델로 학습 및 평가
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_reshaped, y_train_combined, 
                         validation_data=(X_valid_reshaped, y_valid), 
                         epochs=10, 
                         batch_size=32)

# 테스트 데이터 평가
test_loss, test_accuracy = best_model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f"Test Accuracy with Best Model: {test_accuracy * 100:.2f}%")

### **3-2. 랜덤 서치(Randomized Search)**
* 하이퍼파라미터 공간에서 무작위로 일부 조합을 선택하여 탐색하는 방법

![Local Image](Contents/Random_Search.png)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from kerastuner.tuners import RandomSearch

# Keras Tuner에서 사용할 모델 생성 함수
def build_model(hp):
    model = Sequential()
    # 첫 번째 Convolutional Layer
    model.add(Conv2D(
        filters=hp.Choice('filters1', values=[32, 64]),  # 'filters1'에 대해 32 또는 64 선택 #hp.Choice('name', values=[value1, value2, ...])
        kernel_size=(3, 3), 
        activation='relu', 
        input_shape=(28, 28, 1)))
    model.add(Conv2D(
        filters=hp.Choice('filters2', values=[64, 128]), # 'filters2'에 대해 64 또는 128 선택
        kernel_size=(3, 3), 
        activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout1', values=[0.2, 0.3]))) # 'dropout1'에 대해 0.2 또는 0.3 선택

    # 두 번째 Convolutional Layer
    model.add(Conv2D(
        filters=hp.Choice('filters3', values=[128, 256]), 
        kernel_size=(3, 3), 
        activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout2', values=[0.3, 0.4])))

    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(hp.Choice('dense_units', values=[128, 256]), activation='relu'))
    model.add(Dropout(hp.Choice('dropout3', values=[0.4, 0.5])))
    model.add(Dense(10, activation='softmax'))  # 10개의 클래스

    # Optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Keras Tuner Random Search 설정
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',  # 최적화 목표
    max_trials=10,            # 시도할 하이퍼파라미터 조합 수
    executions_per_trial=1,   # 각 조합 실행 횟수
    directory='my_dir',       # 결과 저장 디렉토리
    project_name='cnn_tuning' # 프로젝트 이름
)

# 데이터 준비
X_train_reshaped = X_train_combined.reshape(-1, 28, 28, 1)  # 4D 텐서로 변환
X_valid_reshaped = X_valid.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)

# Random Search 실행
tuner.search(X_train_reshaped, y_train_combined, 
             validation_data=(X_valid_reshaped, y_valid), 
             epochs=10, 
             batch_size=32)

# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# 최적 모델로 학습 및 평가
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_reshaped, y_train_combined, 
                         validation_data=(X_valid_reshaped, y_valid), 
                         epochs=10, 
                         batch_size=32)

# 테스트 데이터 평가
test_loss, test_accuracy = best_model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f"Test Accuracy with Best Model: {test_accuracy * 100:.2f}%")

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define hyperparameter distributions
param_dist = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation': ['relu', 'sigmoid', 'tanh']
}

# Create KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32)

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, cv=3, n_iter=5)
random_search_result = random_search.fit(X_train, y_train)

### **2-3. 베이지안 최적화(Bayesian Optimization)**
* 이전 탐색 결과를 바탕으로, 다음 탐색할 하이퍼파라미터 조합을 선택하는 방법

![Local Image](Contents/Bayesian_Search.png)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from kerastuner.tuners import BayesianOptimization

# Keras Tuner에서 사용할 모델 생성 함수
def build_model(hp):
    model = Sequential()
    # 첫 번째 Convolutional Layer
    model.add(Conv2D(
        filters=hp.Choice('filters1', values=[32, 64]),  # 'filters1'에 대해 32 또는 64 선택
        kernel_size=(3, 3), 
        activation='relu', 
        input_shape=(28, 28, 1)))
    model.add(Conv2D(
        filters=hp.Choice('filters2', values=[64, 128]), # 'filters2'에 대해 64 또는 128 선택
        kernel_size=(3, 3), 
        activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout1', values=[0.2, 0.3]))) # 'dropout1'에 대해 0.2 또는 0.3 선택

    # 두 번째 Convolutional Layer
    model.add(Conv2D(
        filters=hp.Choice('filters3', values=[128, 256]), 
        kernel_size=(3, 3), 
        activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout2', values=[0.3, 0.4])))

    # Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(hp.Choice('dense_units', values=[128, 256]), activation='relu'))
    model.add(Dropout(hp.Choice('dropout3', values=[0.4, 0.5])))
    model.add(Dense(10, activation='softmax'))  # 10개의 클래스

    # Optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd'])
    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.01])
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate, momentum=0.9)

    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Keras Tuner Bayesian Optimization 설정
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',  # 최적화 목표
    max_trials=20,            # 시도할 하이퍼파라미터 조합 수
    num_initial_points=5,     # 초기 랜덤 시도 횟수
    directory='my_dir',       # 결과 저장 디렉토리
    project_name='cnn_tuning_bayes' # 프로젝트 이름
)

# 데이터 준비
X_train_reshaped = X_train_combined.reshape(-1, 28, 28, 1)  # 4D 텐서로 변환
X_valid_reshaped = X_valid.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)

# Bayesian Optimization 실행
tuner.search(X_train_reshaped, y_train_combined, 
             validation_data=(X_valid_reshaped, y_valid), 
             epochs=10, 
             batch_size=32)

# 최적 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps.values}")

# 최적 모델로 학습 및 평가
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train_reshaped, y_train_combined, 
                         validation_data=(X_valid_reshaped, y_valid), 
                         epochs=10, 
                         batch_size=32)

# 테스트 데이터 평가
test_loss, test_accuracy = best_model.evaluate(X_test_reshaped, y_test, verbose=0)
print(f"Test Accuracy with Best Model: {test_accuracy * 100:.2f}%")